In [1]:
import pdb


import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import theano
import pandas as pd
print(f"Running on PyMC3 v{pm.__version__}")
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

Running on PyMC3 v3.11.2


In [2]:
samples = 2500
tune = 2500

In [3]:
# data = pd.read_csv('test_lme_data.csv')
data_clust1_AND = pd.read_csv('FaNoOcc_NeNoOcc_insta_abs_detrend_lme_smile_cluster_beta_data_AND_ori.csv')
# data_clust1.head()
# data = data.iloc[0:7000,]
data_clust1_AND.condition = data_clust1_AND.condition_Smile_auto.str.split("_",n=1, expand = True)[1]
data_clust1_AND.condition = data_clust1_AND.condition.astype('int')

<ipython-input-3-d4e41724a3b0>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_clust1_AND.condition = data_clust1_AND.condition_Smile_auto.str.split("_",n=1, expand = True)[1]


In [4]:
n_dyads = len(data_clust1_AND.Dyad_no_Eye_tracker_gaze.unique())
dyad_idx = np.array(data_clust1_AND.Dyad_no_Eye_tracker_gaze.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_1_AND:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
    feature_est = intercept[dyad_idx]  +  eye_gaze[dyad_idx] * data_clust1_AND.Fun_eyeface_joint.values + mouth_size[dyad_idx] * data_clust1_AND.Fun_mouth_size_joint.values +  mov[dyad_idx] * data_clust1_AND.Fun_ALL_joint.values
#     feature_est = intercept[dyad_idx] 

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust1_AND.chan_freq_data)
#     hierarchical_trace_clust_1_null = pm.sample(5000, tune=5000, target_accept=0.99)
    hierarchical_trace_clust_1_AND = pm.sample(samples, tune=tune, target_accept=0.99)

<ipython-input-4-9dde6c86d92d>:36: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  hierarchical_trace_clust_1_AND = pm.sample(samples, tune=tune, target_accept=0.99)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, mov, mouth_size, eye_gaze, intercept, sigma_mov, mu_mov, sigma_mouth_size, mu_mouth_size, sigma_eye_gaze, mu_eye_gaze, sigma_intercept, mu_intercept]


Sampling 4 chains for 2_500 tune and 2_500 draw iterations (10_000 + 10_000 draws total) took 43878 seconds.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
There were 14 divergences after tuning. Increase `target_accept` or reparameterize.
There were 17 divergences after tuning. Increase `target_accept` or reparameterize.
There were 15 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


In [5]:
#8
hierarchical_trace_clust_1_AND_save = pm.save_trace(hierarchical_trace_clust_1_AND,directory='.model_clust_1_binary_insta_abs_detrend_AND_2500_ori.trace')    

In [6]:
# az.rhat(hierarchical_trace_clust_1_AND)

In [7]:
# data = pd.read_csv('test_lme_data.csv')
data_clust1_XOR = pd.read_csv('FaNoOcc_NeNoOcc_insta_abs_detrend_lme_smile_cluster_beta_data_XOR_ori.csv')
# data_clust1.head()
# data = data.iloc[0:7000,]
data_clust1_XOR.condition = data_clust1_XOR.condition_Smile_auto.str.split("_",n=1, expand = True)[1]
data_clust1_XOR.condition = data_clust1_XOR.condition.astype('int')

<ipython-input-7-d0d069bbd91c>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_clust1_XOR.condition = data_clust1_XOR.condition_Smile_auto.str.split("_",n=1, expand = True)[1]


In [8]:
n_dyads = len(data_clust1_XOR.Dyad_no_Eye_tracker_gaze.unique())
dyad_idx = np.array(data_clust1_XOR.Dyad_no_Eye_tracker_gaze.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_1_XOR:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
    feature_est = intercept[dyad_idx]  +  eye_gaze[dyad_idx] * data_clust1_XOR.Fun_eyeface_joint.values + mouth_size[dyad_idx] * data_clust1_XOR.Fun_mouth_size_joint.values +  mov[dyad_idx] * data_clust1_XOR.Fun_ALL_joint.values
#     feature_est = intercept[dyad_idx] 

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust1_XOR.chan_freq_data)
#     hierarchical_trace_clust_1_null = pm.sample(5000, tune=5000, target_accept=0.99)
    hierarchical_trace_clust_1_XOR = pm.sample(samples, tune=tune, target_accept=0.99)

<ipython-input-8-f90d7f4daa1b>:36: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  hierarchical_trace_clust_1_XOR = pm.sample(samples, tune=tune, target_accept=0.99)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, mov, mouth_size, eye_gaze, intercept, sigma_mov, mu_mov, sigma_mouth_size, mu_mouth_size, sigma_eye_gaze, mu_eye_gaze, sigma_intercept, mu_intercept]


Sampling 4 chains for 2_500 tune and 2_500 draw iterations (10_000 + 10_000 draws total) took 75641 seconds.
There were 20 divergences after tuning. Increase `target_accept` or reparameterize.
There were 146 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.9482377495175797, but should be close to 0.99. Try to increase the number of tuning steps.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 29 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.971694376955983, but should be close to 0.99. Try to increase the number of tuning steps.
The estimated number of effective samples is smaller than 200 for some parameters.


In [9]:
hierarchical_trace_clust_1_XOR_save = pm.save_trace(hierarchical_trace_clust_1_XOR,directory='.model_clust_1_binary_insta_abs_detrend_XOR_2500_ori.trace')    

In [10]:
# hierarchical_trace_clust_1_AND_az_model = az.from_pymc3(hierarchical_trace_clust_1_AND)
# hierarchical_trace_clust_1_XOR_az_model = az.from_pymc3(hierarchical_trace_clust_1_XOR)
# df_comp_loo_1 = az.compare({"AND": hierarchical_trace_clust_1_AND_az_model, "XOR": hierarchical_trace_clust_1_XOR_az_model})
# df_comp_loo_1


In [11]:
# az.rhat(hierarchical_trace_clust_1_XOR)

In [12]:
# data = pd.read_csv('test_lme_data.csv')
data_clust2_AND = pd.read_csv('FaNoOcc_NeNoOcc_insta_abs_detrend_lme_smile_cluster_gamma_data_AND_ori.csv')
# data_clust1.head()
# data = data.iloc[0:7000,]
data_clust2_AND.condition = data_clust2_AND.condition_Smile_auto.str.split("_",n=1, expand = True)[1]
data_clust2_AND.condition = data_clust2_AND.condition.astype('int')

<ipython-input-12-45d9b3f77de3>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_clust2_AND.condition = data_clust2_AND.condition_Smile_auto.str.split("_",n=1, expand = True)[1]


In [13]:
n_dyads = len(data_clust2_AND.Dyad_no_Eye_tracker_gaze.unique())
dyad_idx = np.array(data_clust2_AND.Dyad_no_Eye_tracker_gaze.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_2_AND:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
    feature_est = intercept[dyad_idx]  +  eye_gaze[dyad_idx] * data_clust2_AND.Fun_eyeface_joint.values + mouth_size[dyad_idx] * data_clust2_AND.Fun_mouth_size_joint.values +  mov[dyad_idx] * data_clust2_AND.Fun_ALL_joint.values
#     feature_est = intercept[dyad_idx] 

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust2_AND.chan_freq_data)
#     hierarchical_trace_clust_1_null = pm.sample(5000, tune=5000, target_accept=0.99)
    hierarchical_trace_clust_2_AND = pm.sample(samples, tune=tune, target_accept=0.99)

<ipython-input-13-e015afd12281>:36: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  hierarchical_trace_clust_2_AND = pm.sample(samples, tune=tune, target_accept=0.99)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, mov, mouth_size, eye_gaze, intercept, sigma_mov, mu_mov, sigma_mouth_size, mu_mouth_size, sigma_eye_gaze, mu_eye_gaze, sigma_intercept, mu_intercept]


Sampling 4 chains for 2_500 tune and 2_500 draw iterations (10_000 + 10_000 draws total) took 15559 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


In [14]:
hierarchical_trace_clust_2_AND_save = pm.save_trace(hierarchical_trace_clust_2_AND,directory='.model_clust_2_binary_insta_abs_detrend_AND_2500_ori.trace')    

In [15]:
# az.rhat(hierarchical_trace_clust_2_AND)

In [16]:
# data = pd.read_csv('test_lme_data.csv')
data_clust2_XOR = pd.read_csv('FaNoOcc_NeNoOcc_insta_abs_detrend_lme_smile_cluster_gamma_data_XOR_ori.csv')
# data_clust1.head()
# data = data.iloc[0:7000,]
data_clust2_XOR.condition = data_clust2_XOR.condition_Smile_auto.str.split("_",n=1, expand = True)[1]
data_clust2_XOR.condition = data_clust2_XOR.condition.astype('int')

<ipython-input-16-df49e33887fa>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_clust2_XOR.condition = data_clust2_XOR.condition_Smile_auto.str.split("_",n=1, expand = True)[1]


In [17]:
n_dyads = len(data_clust2_XOR.Dyad_no_Eye_tracker_gaze.unique())
dyad_idx = np.array(data_clust2_XOR.Dyad_no_Eye_tracker_gaze.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_2_XOR:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
    feature_est = intercept[dyad_idx]  +  eye_gaze[dyad_idx] * data_clust2_XOR.Fun_eyeface_joint.values + mouth_size[dyad_idx] * data_clust2_XOR.Fun_mouth_size_joint.values +  mov[dyad_idx] * data_clust2_XOR.Fun_ALL_joint.values
#     feature_est = intercept[dyad_idx] 

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust2_XOR.chan_freq_data)
#     hierarchical_trace_clust_1_null = pm.sample(5000, tune=5000, target_accept=0.99)
    hierarchical_trace_clust_2_XOR = pm.sample(samples, tune=tune, target_accept=0.99)

<ipython-input-17-d6c91c2bef6c>:36: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  hierarchical_trace_clust_2_XOR = pm.sample(samples, tune=tune, target_accept=0.99)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, mov, mouth_size, eye_gaze, intercept, sigma_mov, mu_mov, sigma_mouth_size, mu_mouth_size, sigma_eye_gaze, mu_eye_gaze, sigma_intercept, mu_intercept]


Sampling 4 chains for 2_500 tune and 2_500 draw iterations (10_000 + 10_000 draws total) took 10794 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [18]:
hierarchical_trace_clust_2_XOR_save = pm.save_trace(hierarchical_trace_clust_2_XOR,directory='.model_clust_2_binary_insta_abs_detrend_XOR_2500_ori.trace')    

In [19]:
# az.rhat(hierarchical_trace_clust_2_XOR)

In [20]:
# data = pd.read_csv('test_lme_data.csv')
data_clust3_AND = pd.read_csv('FaNoOcc_NeNoOcc_insta_abs_detrend_lme_smile_cluster_alpha_data_AND_ori.csv')
# data_clust1.head()
# data = data.iloc[0:7000,]
data_clust3_AND.condition = data_clust3_AND.condition_Smile_auto.str.split("_",n=1, expand = True)[1]
data_clust3_AND.condition = data_clust3_AND.condition.astype('int')

<ipython-input-20-c1adeccd9299>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_clust3_AND.condition = data_clust3_AND.condition_Smile_auto.str.split("_",n=1, expand = True)[1]


In [21]:
n_dyads = len(data_clust3_AND.Dyad_no_Eye_tracker_gaze.unique())
dyad_idx = np.array(data_clust3_AND.Dyad_no_Eye_tracker_gaze.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_3_AND:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
    feature_est = intercept[dyad_idx]  +  eye_gaze[dyad_idx] * data_clust3_AND.Fun_eyeface_joint.values + mouth_size[dyad_idx] * data_clust3_AND.Fun_mouth_size_joint.values +  mov[dyad_idx] * data_clust3_AND.Fun_ALL_joint.values
#     feature_est = intercept[dyad_idx] 

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust3_AND.chan_freq_data)
#     hierarchical_trace_clust_1_null = pm.sample(5000, tune=5000, target_accept=0.99)
    hierarchical_trace_clust_3_AND = pm.sample(samples, tune=tune, target_accept=0.99)

<ipython-input-21-cd464ffd103a>:36: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  hierarchical_trace_clust_3_AND = pm.sample(samples, tune=tune, target_accept=0.99)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, mov, mouth_size, eye_gaze, intercept, sigma_mov, mu_mov, sigma_mouth_size, mu_mouth_size, sigma_eye_gaze, mu_eye_gaze, sigma_intercept, mu_intercept]


Sampling 4 chains for 2_500 tune and 2_500 draw iterations (10_000 + 10_000 draws total) took 83926 seconds.
There were 1649 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.4923031636863557, but should be close to 0.99. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 7 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.2 for some parameters.
The estimated number of effective samples is smaller than 200 for some parameters.


In [22]:
hierarchical_trace_clust_3_AND_save = pm.save_trace(hierarchical_trace_clust_3_AND,directory='.model_clust_3_binary_insta_abs_detrend_AND_2500_ori.trace',overwrite=True)    

In [23]:
# data = pd.read_csv('test_lme_data.csv')
data_clust3_XOR = pd.read_csv('FaNoOcc_NeNoOcc_insta_abs_detrend_lme_smile_cluster_alpha_data_XOR_ori.csv')
# data_clust1.head()
# data = data.iloc[0:7000,]
data_clust3_XOR.condition = data_clust3_XOR.condition_Smile_auto.str.split("_",n=1, expand = True)[1]
data_clust3_XOR.condition = data_clust3_XOR.condition.astype('int')

<ipython-input-23-6e00c6b13c97>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_clust3_XOR.condition = data_clust3_XOR.condition_Smile_auto.str.split("_",n=1, expand = True)[1]


In [24]:
n_dyads = len(data_clust3_XOR.Dyad_no_Eye_tracker_gaze.unique())
dyad_idx = np.array(data_clust3_XOR.Dyad_no_Eye_tracker_gaze.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_3_XOR:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
    feature_est = intercept[dyad_idx]  +  eye_gaze[dyad_idx] * data_clust3_XOR.Fun_eyeface_joint.values + mouth_size[dyad_idx] * data_clust3_XOR.Fun_mouth_size_joint.values +  mov[dyad_idx] * data_clust3_XOR.Fun_ALL_joint.values
#     feature_est = intercept[dyad_idx] 

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust3_XOR.chan_freq_data)
#     hierarchical_trace_clust_1_null = pm.sample(5000, tune=5000, target_accept=0.99)
    hierarchical_trace_clust_3_XOR = pm.sample(samples, tune=tune, target_accept=0.99)

<ipython-input-24-3f8e98329b80>:36: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  hierarchical_trace_clust_3_XOR = pm.sample(samples, tune=tune, target_accept=0.99)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, mov, mouth_size, eye_gaze, intercept, sigma_mov, mu_mov, sigma_mouth_size, mu_mouth_size, sigma_eye_gaze, mu_eye_gaze, sigma_intercept, mu_intercept]


Sampling 4 chains for 2_500 tune and 2_500 draw iterations (10_000 + 10_000 draws total) took 62842 seconds.
There were 20 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.9695211862021191, but should be close to 0.99. Try to increase the number of tuning steps.
There were 17 divergences after tuning. Increase `target_accept` or reparameterize.
There were 7 divergences after tuning. Increase `target_accept` or reparameterize.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.


In [25]:
hierarchical_trace_clust_3_XOR_save = pm.save_trace(hierarchical_trace_clust_3_XOR,directory='.model_clust_3_binary_insta_abs_detrend_XOR_2500_ori.trace')    

In [26]:
# hierarchical_trace_clust_3_AND_az_model = az.from_pymc3(hierarchical_trace_clust_3_AND)
# hierarchical_trace_clust_3_XOR_az_model = az.from_pymc3(hierarchical_trace_clust_3_XOR)
# df_comp_loo_3 = az.compare({"AND": hierarchical_trace_clust_3_AND_az_model, "XOR": hierarchical_trace_clust_3_XOR_az_model})
# df_comp_loo_3


In [27]:
# hierarchical_trace_clust_1_AND_az_model = az.from_pymc3(hierarchical_trace_clust_1_AND)

In [28]:
# az.loo(hierarchical_trace_clust_1_AND_az_model,hierarchical_model_clust_1_AND)

In [29]:
# hierarchical_trace_clust_1_XOR_az_model = az.from_pymc3(hierarchical_trace_clust_1_XOR)

In [30]:
# az.loo(hierarchical_trace_clust_1_XOR_az_model,hierarchical_model_clust_1_XOR)

In [31]:
# hierarchical_trace_clust_2_AND_az_model = az.from_pymc3(hierarchical_trace_clust_2_AND)

In [32]:
# az.loo(hierarchical_trace_clust_2_AND_az_model,hierarchical_model_clust_2_AND)

In [33]:
# hierarchical_trace_clust_2_XOR_az_model = az.from_pymc3(hierarchical_trace_clust_2_XOR)

In [34]:
# az.loo(hierarchical_trace_clust_2_XOR_az_model,hierarchical_trace_clust_2_XOR)

In [35]:
# df_comp_loo = az.compare({"AND": hierarchical_trace_clust_1_AND_az_model, "XOR": hierarchical_trace_clust_1_XOR_az_model})
# df_comp_loo

In [36]:
# df_comp_loo_2 = az.compare({"AND": hierarchical_trace_clust_2_AND_az_model, "XOR": hierarchical_trace_clust_2_XOR_az_model})
# df_comp_loo_2

In [37]:
# az.plot_compare(df_comp_loo)

In [38]:
# az.plot_compare(df_comp_loo_2)

In [39]:
# pm.model_graph.model_to_graphviz(hierarchical_model_clust_2_XOR)